In [55]:
from rasa.core.tracker_store import (
    TrackerStore,
    InMemoryTrackerStore,
    MongoTrackerStore)
from rasa.core.events import (
    SlotSet,
    ActionExecuted,
    Restarted,
    UserUttered,
    SessionStarted,
    BotUttered,
    Event,
)
from rasa.core.trackers import DialogueStateTracker
from rasa.core.domain import Domain

def get_default_domain():
    content = """
    actions:
      - utter_hello

    intents:
      - greet
      - bye
      - affirm
      - deny
    entities:
      - object_type
      - mention
      - attribute
      - sents

    slots:
      attribute:
        type: unfeaturized
      mention:
        type: unfeaturized
      object:
        type: unfeaturized
      object_type:
        type: unfeaturized
      parameters:
        type: unfeaturized
      sents:
        type: unfeaturized
    """
    return Domain.from_yaml(content)

default_domain=get_default_domain()
tracker_store = MongoTrackerStore(domain=default_domain)

In [73]:
# tracker_store = tracker_store_type(default_domain, **tracker_store_kwargs)
events = [
    UserUttered("Hola", {"name": "greet"}),
    BotUttered("Hi"),
    SessionStarted(),
    UserUttered("Ciao", {"name": "greet"}),
]
sender_id = "sql1"
tracker = DialogueStateTracker.from_events(sender_id, events)
tracker_store.save(tracker)

# Save other tracker to ensure that we don't run into problems with other senders
other_tracker = DialogueStateTracker.from_events("other-sender", [SessionStarted()])
tracker_store.save(other_tracker)

# Retrieve tracker with events since latest SessionStarted
tracker = tracker_store.retrieve(sender_id)
tracker.events

deque([<rasa.core.events.SessionStarted at 0x1224206a0>,
       <rasa.core.events.UserUttered at 0x12217f128>])

In [75]:
tracker_store.conversations.drop()

In [17]:
tracker.events

deque([<rasa.core.events.SessionStarted at 0x120abfda0>,
       <rasa.core.events.UserUttered at 0x120bac6a0>])

In [77]:
# Create tracker with a SessionStarted event
events = [    
#     SessionStarted(),
    UserUttered("Hola", {"name": "greet"}),
    BotUttered("Hi"),
    UserUttered("Ciao", {"name": "greet"}),
    BotUttered("Hi2"),
]

sender_id = "sql4"
tracker = DialogueStateTracker.from_events(sender_id, events)
tracker_store.save(tracker)

# Save other tracker to ensure that we don't run into problems with other senders
other_tracker = DialogueStateTracker.from_events('other_id', [SessionStarted()])
tracker_store.save(other_tracker)

tracker = tracker_store.retrieve(sender_id)
tracker.events

deque([<rasa.core.events.UserUttered at 0x1224319e8>,
       BotUttered('Hi', {}, {}, 1583519619.381093),
       BotUttered('Hi2', {}, {}, 1583519619.381098)])

In [78]:
slot_key = "object"
slot_val = "Easter Island"
# tracker = tracker_store.get_or_create_tracker('samlet')
ev = SlotSet(slot_key, slot_val)
tracker.update(ev)
tracker_store.save(tracker)

In [38]:
tracker.current_slot_values()

{'attribute': None,
 'mention': 'second',
 'object': 'Easter Island',
 'object_type': None,
 'parameters': None,
 'sents': None}

In [81]:
again = tracker_store.get_or_create_tracker(sender_id)
again.get_slot(slot_key)

'Easter Island'

In [12]:
# other_tracker = DialogueStateTracker.from_events(sender_id, [SlotSet('mention', 'first')])
# tracker_store.save(other_tracker)

In [80]:
sender_id

'sql4'

In [79]:
tracker = tracker_store.retrieve(sender_id)
tracker.events

deque([<rasa.core.events.UserUttered at 0x12242e5f8>,
       BotUttered('Hi', {}, {}, 1583519619.381093),
       BotUttered('Hi2', {}, {}, 1583519619.381098),
       <rasa.core.events.SlotSet at 0x122431828>])

In [50]:
tracker = tracker_store.get_or_create_tracker('dev')
# tracker.events
dialogue = tracker.as_dialogue()
dialogue.as_dict()

{'events': [{'event': 'action',
   'timestamp': 1583517978.615391,
   'name': 'action_listen',
   'policy': None,
   'confidence': None}],
 'name': 'dev'}

In [53]:
slot_key = "object"
slot_val = "Easter Island (new)"
# tracker = tracker_store.get_or_create_tracker('samlet')
ev = SlotSet(slot_key, slot_val)
tracker.update(ev)
tracker_store.save(tracker)

In [54]:
again = tracker_store.get_or_create_tracker('dev')
again.get_slot(slot_key)

'Easter Island (new)'

In [83]:
# preqs: 
# $ saai talk 'hello' samlet
# $ saai talk '/dump_info{"object":"rr"}' samlet
# config:
# sagas-ai/bots/agent_dispatcher/endpoints.yml
# tracker_store:
#     type: mongod
tracker = tracker_store.get_or_create_tracker('samlet')
# tracker.events
dialogue = tracker.as_dialogue()
dialogue.as_dict()

{'events': [{'event': 'action',
   'timestamp': 1583520256.0032,
   'name': 'action_listen',
   'policy': None,
   'confidence': None},
  {'event': 'user',
   'timestamp': 1583520312.679762,
   'text': '/dump_info{"object":"rr"}',
   'parse_data': {'text': '/dump_info{"object":"rr"}',
    'intent': {'name': 'dump_info', 'confidence': 1.0},
    'intent_ranking': [{'name': 'dump_info', 'confidence': 1.0}],
    'entities': [{'entity': 'object', 'start': 10, 'end': 25, 'value': 'rr'}]},
   'input_channel': 'rest',
   'message_id': '131af204087f4b6e8eb40ced2d74a98d',
   'metadata': {}},
  {'event': 'slot',
   'timestamp': 1583520312.679787,
   'name': 'object',
   'value': 'rr'},
  {'event': 'action',
   'timestamp': 1583520312.904442,
   'name': 'action_dump_info',
   'policy': 'policy_2_MappingPolicy',
   'confidence': 1},
  {'event': 'action',
   'timestamp': 1583520312.911701,
   'name': 'action_listen',
   'policy': 'policy_2_MappingPolicy',
   'confidence': 1},
  {'event': 'user',
   

In [84]:
tracker.current_slot_values()

{'attribute': None,
 'mention': None,
 'object': 'rr',
 'object_type': None,
 'parameters': None,
 'sents': None}